<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/RandomForest/RandomForest_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classification by Random Forest and Count Vector**

----------

 import library

  إستيراد المكتبات

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

 استيراد مجموعات البيانات

--------

**1- Import Data set**

In [ ]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")

------
shuffle  
بعثرة البيانات

In [ ]:
df=shuffle(df)
df

,text,targe
59843,صنف تقرير جديد لشركة ماستر كارد لبطاقات الائتم...,2
23705,أخبارنا المغربية أفادت المذكرة الإخبارية للمند...,2
12729,أخبارنا المغربية تستضيف العاصمة القطرية الدوحة...,3
944,أخبارنا المغربية سناء الوردي بعدما استبشر المغ...,0
3145,علمت الصباح »، أن قناة ميدي تي في بصدد تصوير ا...,0
...,...,...
29650,صوفيا غزال أخبارنا المغربية كمال العماري عضو ي...,3
5317,شرعت فرقة أمنية تابعة للفرقة الوطنية للشرطة ال...,1
665,الممثل بخاري قال إن المخرجين اعتادوا أدوار الف...,0
24343,كشف المهندس إبراهيم جمال أبو عميرة رئيس مجلس إ...,2


------
**2- Clean Data**

تنظيف البيانات

In [ ]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

378
0
text     True
targe    True
dtype: bool
68690
                                                    text  targe
59843  صنف تقرير جديد لشركه ماستر كارد لبطاقات الائتم...      2
23705  اخبارنا المغربيه افادت المذكره الاخباريه لمندو...      2
12729  اخبارنا المغربيه تستضيف العاصمه القطريه الدوحه...      3
944    اخبارنا المغربيه سناء الوردي بعدما استبشر المغ...      0
3145   علمت الصباح » ان قناه ميدي تي في بصد تصوير الج...      0
...                                                  ...    ...
29650  صوفيا غزال اخبارنا المغربيه كمال العماري عضو ي...      3
5317   شرعت فرقه امنيه تابعه لفرقه الوطنيه لشرطه القض...      1
665    مثل بخاري قال ان المخرجين اعتادوا ادوار الفاست...      0
24343  كشف المهندس ابراهيم جمال ابو عميره رئيس مجلس ا...      2
42318  فاز ثلاثه ابناك مغربيه وهي التجاري وفا بنك وصن...      2

[68312 rows x 2 columns]


---------
accuracy calculation function

عمل تابع لحساب الدقة

In [ ]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

----------------------
**3- Feature Extraction:** CountVector

استخراج الميزات(تحويل النص إلى أرقام)

**4- classification:** Random Forest

تصنيف البيانات

with Cross validation

لتوزيع عينات الاختبار والتأكد من عدم وجود

Overfitting

In [ ]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
X1 =clean_df['text'].values
y1 = clean_df['targe'].values
vectorizer = CountVectorizer(max_features=1500)


strtfdKFold = StratifiedKFold(n_splits=5)
kfold = strtfdKFold.split(X1, y1)
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
result_RandomForestClassifier=[]
#
i=0
for k, (train_index, test_index) in enumerate(kfold):
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y1[train_index], y1[test_index]
    vectorizer.fit(X_train)
    X_train = vectorizer.transform(X_train)
    X_test  = vectorizer.transform(X_test)
    classifier.fit(X_train, y_train)
    y_pred=(classifier.predict(X_test))
    result_RandomForestClassifier.append(calculate_results(y_test, y_pred))
    i+=1
    print(i)
result_RandomForestClassifier

1
2
3
4
5
CPU times: user 2h 55min 4s, sys: 31.3 s, total: 2h 55min 35s
Wall time: 2h 55min 31s


حساب متوسط النتائج للـ5 مراحل من

 cross validation 

In [ ]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_RandomForestClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_RandomForestClassifier)
f1=f1/len(result_RandomForestClassifier)
precision=precision/len(result_RandomForestClassifier)
recall=recall/len(result_RandomForestClassifier)
# (result_DecisionTreeClassifier)
results_RandomForestClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_RandomForestClassifier

[{'accuracy': 92.69381719677504,
  'f1': 0.9269501239060212,
  'precision': 0.9270895737824528,
  'recall': 0.9269381719677504}]

--------
-----
---------
**5- Show Results**

النتائج

نتائج التدريب لكل قاعدة بيانات

In [ ]:
print('RandomForestClassifier results in dataset 1:')
results_RandomForestClassifier

In [ ]:
print('RandomForestClassifier results in dataset 2:')
results_RandomForestClassifier

In [ ]:
print('RandomForestClassifier results in dataset 3:')
results_RandomForestClassifier

In [ ]:
print('RandomForestClassifier results in dataset 4:')
results_RandomForestClassifier

RandomForestClassifier results in dataset 4:


[{'accuracy': 92.69381719677504,
  'f1': 0.9269501239060212,
  'precision': 0.9270895737824528,
  'recall': 0.9269381719677504}]

In [2]:
Final_Result=[
  {'accuracy': 94.73533651964264,
  'f1': 0.9473993504427642,
  'precision': 0.9476771552006795,
  'recall': 0.9473533651964263}       
  ,
  {'accuracy': 94.24720740589889,
  'f1': 0.942490382060177,
  'precision': 0.9426596506217448,
  'recall': 0.9424720740589889}
  ,
  {'accuracy': 93.08499155869795,
  'f1': 0.9308805430780541,
  'precision': 0.9310418538397369,
  'recall': 0.9308499155869796}
  ,
  {'accuracy': 92.69381719677504,
  'f1': 0.9269501239060212,
  'precision': 0.9270895737824528,
  'recall': 0.9269381719677504}  
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
print("RAM: 2.1 , CPU times: user 2h 55min 4s")
print('Final_Result:')
(Final_Result)

RAM: 2.1 , CPU times: user 2h 55min 4s
Final_Result:


[{'accuracy': 93.69033817025364,
  'f1': 0.9369300998717541,
  'precision': 0.9371170583611536,
  'recall': 0.9369033817025364}]